In [1]:
Today = '31.12.2023'

Data_file = 'sapicdeal31122023' + '.xlsx'
Sheet_in_data_file = 'Export Worksheet'

from datetime import date

Output_file = "_".join([str(date.today()),'IN_SUEK_Deals.xlsx'])
Sheet_in_output_file = 'Deals'

In [2]:
# !dir

In [3]:
import pandas as pd
import numpy as np

data_export = pd.read_excel(Data_file, sheet_name = Sheet_in_data_file)

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period

In [6]:
day = pd.to_datetime(Today)
data_work = data_export

data_work = data_work.query("dealStatusName != 'Cancelled' & invoiced == 'Y' & deliveryPeriodTo > @day").reset_index(drop=True)

data_work[['sellerName', 'buyerName']] = data_work[['sellerName', 'buyerName']].fillna('')
data_work['Counterparty'] = data_work['sellerName'] + data_work['buyerName']

data_work.loc[data_work['actualQuantity'].isna(), 'actualQuantity'] = data_work.loc[data_work['actualQuantity'].isna(), 'BLTonnage']

data_work['Sum_money'] = data_work['actualQuantity'] * data_work['price']

data_work.loc[data_work.dealTypeName == 'Purchasing Deal', 'actualQuantity'] = data_work.actualQuantity.abs()
data_work.loc[data_work.dealTypeName == 'Purchasing Deal', 'Sum_money'] = -1 * data_work.Sum_money.abs()

data_work.loc[data_work.dealTypeName == 'Sales Deal', 'actualQuantity'] = -1 * data_work.actualQuantity.abs()
data_work.loc[data_work.dealTypeName == 'Sales Deal', 'Sum_money'] = data_work.Sum_money.abs()

data_work['CompCode'] = merge_Mapping(data_work, col='companyName')
data_work['Business_segment'] = merge_SalesUnits(data_work, col='CompCode', merge_col='ocpSegment')

data_work['Cpty_Code'] = merge_Mapping(data_work, col='Counterparty')
data_work['Cpty_segment'] = merge_SalesUnits(data_work, col='Cpty_Code', merge_col='ocpSegment').fillna('External')

data_work = Period(data_work, day_for_count=Today, col_with_date='deliveryPeriodTo')

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15112\158127040.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  day = pd.to_datetime(Today)


C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:40: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  day = pd.to_datetime(day_for_count)


In [7]:
data_for_print = data_work[['productTypeName', 'companyName', 'Business_segment', 'Counterparty',  'Cpty_segment', 'currency', 'actualQuantity', 'Sum_money', 'deliveryPeriodTo', 'Days', 'Period']]
data_for_print.to_excel(Output_file, sheet_name=Sheet_in_output_file, index=False)

In [8]:

data_for_print[( data_for_print.Cpty_segment == 'External')][['Counterparty', 'Cpty_segment']].drop_duplicates()

,Counterparty,Cpty_segment
290,SPMK 197 LLC,External


In [ ]:
data_for_print[(data_for_print.Business_segment == 'External')]

In [9]:
data_for_print

,productTypeName,companyName,Business_segment,Counterparty,Cpty_segment,currency,actualQuantity,Sum_money,deliveryPeriodTo,Days,Period
0,Sized coal,SUEK AG (Swiss),SUEK INT,SUEK Polska Sp. z o.o.,SUEK INT,PLN,4500.000,3.710295e+06,2021-11-30,-761,<1M
1,Sized coal,SUEK AG (Swiss),SUEK INT,SUEK Polska Sp. z o.o.,SUEK INT,PLN,1000.000,3.874700e+05,2021-11-30,-761,<1M
2,Sized coal,SUEK AG (Swiss),SUEK INT,SUEK Polska Sp. z o.o.,SUEK INT,PLN,3500.000,2.352665e+06,2021-11-30,-761,<1M
3,Sized coal,SUEK AG (Swiss),SUEK INT,SUEK Polska Sp. z o.o.,SUEK INT,PLN,5500.000,2.001505e+06,2021-11-30,-761,<1M
4,Sized coal,SUEK AG (Swiss),SUEK INT,SUEK Polska Sp. z o.o.,SUEK INT,PLN,4500.000,2.454930e+06,2021-11-30,-761,<1M
...,...,...,...,...,...,...,...,...,...,...,...
749,Iron ore concentrate,SST Shanghai,SUEK INT,SUEK AG,SUEK INT,USD,22300.000,3.205402e+06,2022-06-30,-549,<1M
750,Steam coal,Black Sand Commodities,SUEK INT,SUEK Shanghai Trading Co. Ltd.,SUEK INT,USD,12037.650,1.402507e+06,2022-07-31,-518,<1M
751,Brown coal,SST Shanghai,SUEK INT,SUEK AG,SUEK INT,USD,857.000,3.592544e+04,2021-09-30,-822,<1M
752,Steam coal,Black Sand Commodities,SUEK INT,SUEK Shanghai Trading Co. Ltd.,SUEK INT,USD,9600.000,3.307200e+05,2022-07-31,-518,<1M
